In [1]:
!pip install -r requirements.txt


In [3]:
# Library import convention
import requests
from bs4 import BeautifulSoup

In [24]:
#will go through each page of the webpage and take all the values from the tables.
def scrap_multi_page(URL,total_pages): #Will extract the data

  outer_data_list = []

  for i in range(0,total_pages):
    current_url= URL + str(i)

    # Request the target website for its content and get a response back
    response = requests.get(current_url)

    # A 200 response status means a successful response
    #print(response)

    # Get page content in the form of HTML using Beautiful Soup's HTML parser
    soup =  BeautifulSoup(response.content,'html.parser')

    # Find all data under the tag 'table'
    table_data = soup.find_all('table')

    # Filter and remove the empty strings in the list
    list(filter(None, table_data[0].text.splitlines()))

    # Looping through table_data
    for data in table_data:

      # Get filtered value without empty strings
      value = list(filter(None, data.text.splitlines()))

      # Append each day values to the list
      outer_data_list.append(value)
  return outer_data_list

*Without* cleaning the data, it would be really difficult to load it into the database. Let us transform the data and change it into a suitable format.

The text contains a lot of new line strings denoted by \n. So we will be filtering them to get only the required data.

In [5]:
# returns a new list without any non desired extra values,
def remove_extra(data_list):
  new_data_list=[]
  for i in range(0,len(data_list)):
    new_data_list.append([])
    for j in range(0,len(data_list[i])):
      if(data_list[i][j]=='\u200b \t' or data_list[i][j]=='\u200b \t\u200b \t'): # some cells in the table had hidden values that needed to be excluded.
        print("found and excluded extra value")
        continue
      else: new_data_list[i].append(data_list[i][j])
  return new_data_list

In [27]:
#Returns a list containing list where all the sub list are equivalent to rows from the table
def group_into_rows( data_list,num_columns,start ): #Removing any empty space at the start and end of the strings with strip and grouping up the data so that every sub list is a row for pandas dataframe.
  full_list=[]
  for n in range(0,len(data_list)):
    for i in range(start,len(data_list[n]),num_columns):
      inner_list = []
      for j in range(0,num_columns):
        inner_list.append(data_list[n][i].strip())
        i+=1
      full_list.append(inner_list)
  return full_list

In [17]:
#def group_into_rows( data_list ): #Removing any empty space at the start and end of the strings with strip and grouping up the data so that every sub list is a row for pandas dataframe.
  full_list=[]
  for n in range(0,len(data_list)):
    for i in range(7,len(data_list[n]),5):
      inner_list = []
      for j in range(0,5):
        inner_list.append(data_list[n][i].strip())
        i+=1
      full_list.append(inner_list)
  return full_list

IndentationError: unexpected indent (<ipython-input-17-96e4d66c8353>, line 2)

In [28]:
# URL of the target website
url = "https://aflcio.org/paywatch/highest-paid-ceos?page="

data = scrap_multi_page(url,199)

data1 = remove_extra(data)

data2= group_into_rows( data1,5 ,7)
print(data2)

url = 'https://aflcio.org/paywatch/company-pay-ratios?combine=&industry=All&state=All&sp500=0&page='

data = scrap_multi_page(url,122)

data1 = remove_extra(data)

data3= group_into_rows( data1,4 ,6)
print(data3)

found and excluded extra value
found and excluded extra value
found and excluded extra value
found and excluded extra value
found and excluded extra value
[['BX', 'Blackstone Inc.', 'Stephen Schwarzman', '2022', '$253,122,146'], ['GOOG', 'Alphabet Inc.', 'Sundar Pichai', '2022', '$225,985,145'], ['HTZ', 'Hertz Global Holdings, Inc.', 'Stephen Scherr', '2022', '$182,136,137'], ['PTON', 'Peloton Interactive, Inc.', 'Barry Mccarthy', '2022', '$168,073,420'], ['PANW', 'Palo Alto Networks Inc.', 'Nikesh Arora', '2023', '$151,425,203'], ['LYV', 'Live Nation Entertainment, Inc.', 'Michael Rapino', '2022', '$139,005,565'], ['SRPT', 'Sarepta Therapeutics, Inc.', 'Douglas Ingram', '2022', '$124,938,694'], ['PINS', 'Pinterest, Inc.', 'Bill Ready', '2022', '$122,651,735'], ['LAW', 'CS Disco, Inc.', 'Kiwi Camara', '2022', '$109,531,440'], ['WDAY', 'Workday, Inc.', 'Aneel Bhusri', '2023', '$102,685,309'], ['AAPL', 'Apple Inc.', 'Tim Cook', '2022', '$99,420,097'], ['S', 'Sentinelone, Inc.', 'Tomer We

In [29]:
import pandas as pd
import numpy as np


In [121]:

# Create the pandas DataFrame for them ceo information
df = pd.DataFrame(data2, columns=['Ticker', 'Company','CEO','Year','CEO_Pay'])
df

,Ticker,Company,CEO,Year,CEO_Pay
0,BX,Blackstone Inc.,Stephen Schwarzman,2022,"$253,122,146"
1,GOOG,Alphabet Inc.,Sundar Pichai,2022,"$225,985,145"
2,HTZ,"Hertz Global Holdings, Inc.",Stephen Scherr,2022,"$182,136,137"
3,PTON,"Peloton Interactive, Inc.",Barry Mccarthy,2022,"$168,073,420"
4,PANW,Palo Alto Networks Inc.,Nikesh Arora,2023,"$151,425,203"
...,...,...,...,...,...
3975,CLOV,"Clover Health Investments, Corp.",Vivek Garipalli,2022,$0
3976,PSMT,Pricesmart Inc.,,,$0
3977,RDUS,"Radius Health, Inc.",N/A,,$0
3978,RPRX,Royalty Pharma PLC,Pablo Legorreta,,$0


In [122]:
# Create the pandas DataFrame for the employee information
df2 = pd.DataFrame(data3, columns=['Ticker', 'Company','Median_Worker_Pay','Pay_Ratio'])
df2

,Ticker,Company,Median_Worker_Pay,Pay_Ratio
0,NUS,"NU Skin Enterprises, Inc.",$506,"9,416:1"
1,LYV,"Live Nation Entertainment, Inc.","$25,673","5,414:1"
2,HTZ,"Hertz Global Holdings, Inc.","$36,683","4,983:1"
3,WDC,Western Digital Corp.,"$9,644","3,332:1"
4,ANF,Abercrombie & Fitch CO,"$3,383","3,262:1"
...,...,...,...,...
2419,BOX,Box Inc.,"$195,638",1:1
2420,APP,Applovin Corp.,"$139,502",1:1
2421,BOX,Box Inc.,"$195,638",1:1
2422,FSP,Franklin Street Properties Corp /Ma/,"$205,593",1:1


Cleaning up the data


In [123]:
df['CEO_Pay'] = df['CEO_Pay'].str.replace(',', '')
df['CEO_Pay'] = df['CEO_Pay'].str.replace('$', '')

df['Company'] = df['Company'].str.replace(',', '')

df2['Median_Worker_Pay'] = df2['Median_Worker_Pay'].str.replace(',', '')
df2['Median_Worker_Pay'] = df2['Median_Worker_Pay'].str.replace('$', '')

df2['Company'] = df2['Company'].str.replace(',', '')


<ipython-input-123-d6861206c03c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['CEO_Pay'] = df['CEO_Pay'].str.replace('$', '')
<ipython-input-123-d6861206c03c>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['Median_Worker_Pay'] = df2['Median_Worker_Pay'].str.replace('$', '')


In [124]:
#merging the tables together
df3 = df.merge(df2, how='inner', on= 'Company')
df3

,Ticker_x,Company,CEO,Year,CEO_Pay,Ticker_y,Median_Worker_Pay,Pay_Ratio
0,BX,Blackstone Inc.,Stephen Schwarzman,2022,253122146,BX,237000,"1,068:1"
1,GOOG,Alphabet Inc.,Sundar Pichai,2022,225985145,GOOG,279802,808:1
2,HTZ,Hertz Global Holdings Inc.,Stephen Scherr,2022,182136137,HTZ,36683,"4,983:1"
3,PTON,Peloton Interactive Inc.,Barry Mccarthy,2022,168073420,PTON,73117,"2,299:1"
4,PANW,Palo Alto Networks Inc.,Nikesh Arora,2023,151425203,PANW,206035,735:1
...,...,...,...,...,...,...,...,...
2420,CNNE,Cannae Holdings Inc.,Richard Massey,2022,1,CNNE,7963,96:1
2421,PSMT,Pricesmart Inc.,,,0,PSMT,9775,"1,042:1"
2422,RDUS,Radius Health Inc.,N/A,,0,RDUS,248946,14:1
2423,RDUS,Radius Health Inc.,N/A,,0,RDUS,248946,14:1


In [125]:
df3=df3.drop_duplicates()

In [126]:
df3=df3.drop(columns=['Ticker_y'])

In [127]:
df3=df3.rename(columns={"Ticker_x": "Ticker"})

In [128]:
df3

,Ticker,Company,CEO,Year,CEO_Pay,Median_Worker_Pay,Pay_Ratio
0,BX,Blackstone Inc.,Stephen Schwarzman,2022,253122146,237000,"1,068:1"
1,GOOG,Alphabet Inc.,Sundar Pichai,2022,225985145,279802,808:1
2,HTZ,Hertz Global Holdings Inc.,Stephen Scherr,2022,182136137,36683,"4,983:1"
3,PTON,Peloton Interactive Inc.,Barry Mccarthy,2022,168073420,73117,"2,299:1"
4,PANW,Palo Alto Networks Inc.,Nikesh Arora,2023,151425203,206035,735:1
...,...,...,...,...,...,...,...
2419,URBN,Urban Outfitters Inc.,Richard Hayne,2023,55462,16378,3:1
2420,CNNE,Cannae Holdings Inc.,Richard Massey,2022,1,7963,96:1
2421,PSMT,Pricesmart Inc.,,,0,9775,"1,042:1"
2422,RDUS,Radius Health Inc.,N/A,,0,248946,14:1


In [105]:
df3.tail(10)

,Ticker,Company,CEO,Year,CEO_Pay,Median_Worker_Pay,Pay_Ratio
2414,UIHC,United Insurance Holdings Corp.,Daniel Peed,2022,161900,95972,2:1
2415,IDR,Idaho Strategic Resources Inc.,John Swallow,2022,138918,95189,1:1
2416,TTWO,Take Two Interactive Software Inc.,Strauss Zelnick,2023,115015,75276,578:1
2417,APP,Applovin Corp.,Adam Foroughi,2022,104355,139502,1:1
2418,TTEC,Ttec Holdings Inc.,Kenneth Tuchman,2022,69683,10008,7:1
2419,URBN,Urban Outfitters Inc.,Richard Hayne,2023,55462,16378,3:1
2420,CNNE,Cannae Holdings Inc.,Richard Massey,2022,1,7963,96:1
2421,PSMT,Pricesmart Inc.,,,0,9775,"1,042:1"
2422,RDUS,Radius Health Inc.,N/A,,0,248946,14:1
2424,RPRX,Royalty Pharma PLC,Pablo Legorreta,,0,337085,277:1


We have only 3 rows with missing values or what seems incorrect data (such as the Cannae holding inc CEO with a salary of 1$), so i remove those ones.

In [129]:
df3= df3.drop([2420,2421,2422,2424])

In [130]:
df3

,Ticker,Company,CEO,Year,CEO_Pay,Median_Worker_Pay,Pay_Ratio
0,BX,Blackstone Inc.,Stephen Schwarzman,2022,253122146,237000,"1,068:1"
1,GOOG,Alphabet Inc.,Sundar Pichai,2022,225985145,279802,808:1
2,HTZ,Hertz Global Holdings Inc.,Stephen Scherr,2022,182136137,36683,"4,983:1"
3,PTON,Peloton Interactive Inc.,Barry Mccarthy,2022,168073420,73117,"2,299:1"
4,PANW,Palo Alto Networks Inc.,Nikesh Arora,2023,151425203,206035,735:1
...,...,...,...,...,...,...,...
2415,IDR,Idaho Strategic Resources Inc.,John Swallow,2022,138918,95189,1:1
2416,TTWO,Take Two Interactive Software Inc.,Strauss Zelnick,2023,115015,75276,578:1
2417,APP,Applovin Corp.,Adam Foroughi,2022,104355,139502,1:1
2418,TTEC,Ttec Holdings Inc.,Kenneth Tuchman,2022,69683,10008,7:1


We change the data type of Median worker pay and CEO pay to integer

In [131]:
df3['CEO_Pay'] = df3.CEO_Pay.astype(int)
df3['Median_Worker_Pay'] = df3.Median_Worker_Pay.astype(int)

In [132]:
df3

,Ticker,Company,CEO,Year,CEO_Pay,Median_Worker_Pay,Pay_Ratio
0,BX,Blackstone Inc.,Stephen Schwarzman,2022,253122146,237000,"1,068:1"
1,GOOG,Alphabet Inc.,Sundar Pichai,2022,225985145,279802,808:1
2,HTZ,Hertz Global Holdings Inc.,Stephen Scherr,2022,182136137,36683,"4,983:1"
3,PTON,Peloton Interactive Inc.,Barry Mccarthy,2022,168073420,73117,"2,299:1"
4,PANW,Palo Alto Networks Inc.,Nikesh Arora,2023,151425203,206035,735:1
...,...,...,...,...,...,...,...
2415,IDR,Idaho Strategic Resources Inc.,John Swallow,2022,138918,95189,1:1
2416,TTWO,Take Two Interactive Software Inc.,Strauss Zelnick,2023,115015,75276,578:1
2417,APP,Applovin Corp.,Adam Foroughi,2022,104355,139502,1:1
2418,TTEC,Ttec Holdings Inc.,Kenneth Tuchman,2022,69683,10008,7:1


In [133]:
df3.dtypes

Ticker               object
Company              object
CEO                  object
Year                 object
CEO_Pay               int64
Median_Worker_Pay     int64
Pay_Ratio            object
dtype: object

In [134]:
df3.to_csv('CEO_DIMENSION_TABLE.csv', index=False)

In [137]:
!pip install chdb
import chdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 MB 1.8 MB/s eta 0:00:00


In [138]:

import chdb.dataframe as cdf
import pandas as pd

ceo =pd.read_csv("/content/CEO_DIMENSION_TABLE.csv")

query_sql = """
select
  Ticker, CEO, CEO_Pay, Median_Worker_Pay, CEO_Pay/Median_Worker_Pay * 100 as ratio_x, Pay_Ratio
  from __ceo__
"""

res = cdf.query(sql=query_sql,ceo=ceo)

print(res)

Please upgrade pandas to version 2.0.0 or higher to have better performance
       Ticker                    CEO    CEO_Pay  Median_Worker_Pay  \
0       b'BX'  b'Stephen Schwarzman'  253122146             237000   
1     b'GOOG'       b'Sundar Pichai'  225985145             279802   
2      b'HTZ'      b'Stephen Scherr'  182136137              36683   
3     b'PTON'      b'Barry Mccarthy'  168073420              73117   
4     b'PANW'        b'Nikesh Arora'  151425203             206035   
...       ...                    ...        ...                ...   
2171   b'IDR'        b'John Swallow'     138918              95189   
2172  b'TTWO'     b'Strauss Zelnick'     115015              75276   
2173   b'APP'       b'Adam Foroughi'     104355             139502   
2174  b'TTEC'     b'Kenneth Tuchman'      69683              10008   
2175  b'URBN'       b'Richard Hayne'      55462              16378   

            ratio_x   Pay_Ratio  
0     106802.593249  b'1,068:1'  
1      80766.09

Loading the dataframe into clickhouse

In [141]:
import clickhouse_connect

timeframe_in = "D1"  # timeframe for neural network training - input and we will trade on the same timeframe

CLICKHOUSE_CLOUD_HOSTNAME = 'csi4142-clickhouse.mershab.xyz'
CLICKHOUSE_CLOUD_USER = 'csi'
CLICKHOUSE_CLOUD_PASSWORD = 'iceberg'
TABLE_NAME = 'ceo_dimension_table_test'


In [158]:
client = clickhouse_connect.get_client(host=CLICKHOUSE_CLOUD_HOSTNAME, port=443, user=CLICKHOUSE_CLOUD_USER, password=CLICKHOUSE_CLOUD_PASSWORD,database='default')

# Check if table exists and create if it doesn't
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    Ticker String,
    Company String,
    CEO String,
    Year UInt32,
    CEO_Pay UInt32,
    Median_Worker_Pay UInt32,
    Pay_Ratio String
) ENGINE = MergeTree()
ORDER BY CEO_Pay
"""

#client.command(f'DROP TABLE IF EXISTS {TABLE_NAME}')
client.command(create_table_query)
print(client.ping())
print("connected to " + CLICKHOUSE_CLOUD_HOSTNAME + "\n")

True
connected to csi4142-clickhouse.mershab.xyz



In [159]:
client.insert_df(TABLE_NAME, df3)